In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import mysql.connector
import pandas as pd


In [ ]:
def connect_to_db():
    return mysql.connector.connect(
       host="localhost",
       user="root",
       password="",
       database="ems"
    )

In [ ]:
import pandas as pd
import mysql.connector



def first_feature():
    try:
        conn = connect_to_db()
        cursor = conn.cursor(dictionary=True)

        query = """
       WITH RECURSIVE all_dates AS (
    SELECT DATE('2024-01-01') AS dt
    UNION ALL
    SELECT dt + INTERVAL 1 DAY
    FROM all_dates
    WHERE dt + INTERVAL 1 DAY <= '2025-03-31'
),
valid_dates_vskp AS (
    SELECT dt
    FROM all_dates
    WHERE DAYOFWEEK(dt) != 1  -- Excluding Sundays
      AND dt NOT IN (SELECT `date` FROM holiday)
),
valid_dates_ggm AS (
    SELECT dt
    FROM all_dates
    WHERE DAYOFWEEK(dt) != 1  -- Excluding Sundays
      AND dt NOT IN (SELECT `date` FROM holiday_ggm)
),
attendance_vskp AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
attendance_ggm AS (
    SELECT 
        UserID,
        DATE(AttendanceTime) AS att_date
    FROM CamsBiometricAttendance_GGM
    WHERE AttendanceType IN ('CheckIn', 'CheckOut')
    GROUP BY UserID, DATE(AttendanceTime)
    HAVING COUNT(DISTINCT AttendanceType) = 2
),
leave_days AS (
    -- First, expand leave periods into individual leave dates
    WITH RECURSIVE leave_expanded AS (
        SELECT 
            l.empname,
            e.work_location,
            GREATEST(l.`from`, DATE('2024-01-01')) AS leave_date,
            LEAST(l.`to`, DATE('2025-03-31')) AS leave_end
        FROM leaves l
        JOIN emp e ON e.empname = l.empname
        WHERE 
            l.status = 1
            AND l.leavetype IN ('WORK FROM HOME', 'ON DUTY', 'OFFICIAL LEAVE')
            AND l.`to` >= '2024-01-01'
            AND l.`from` <= '2025-03-31'
        
        UNION ALL
        
        SELECT 
            le.empname,
            le.work_location,
            le.leave_date + INTERVAL 1 DAY,
            le.leave_end
        FROM leave_expanded le
        WHERE le.leave_date + INTERVAL 1 DAY <= le.leave_end
    ),
    -- Now filter out Sundays and holidays based on location
    valid_leave_days AS (
        SELECT 
            le.empname,
            le.leave_date
        FROM leave_expanded le
        WHERE DAYOFWEEK(le.leave_date) != 1  -- Exclude Sundays
          AND (
              (le.work_location = 'Visakhapatnam' AND le.leave_date NOT IN (SELECT `date` FROM holiday)) OR
              (le.work_location = 'Gurugram' AND le.leave_date NOT IN (SELECT `date` FROM holiday_ggm))
          )
    )
    SELECT empname, COUNT(*) AS leave_days
    FROM valid_leave_days
    GROUP BY empname
)

,
present_days AS (
    SELECT 
        e.empname,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN (
                SELECT COUNT(*)
                FROM attendance_vskp a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            WHEN e.work_location = 'Gurugram' THEN (
                SELECT COUNT(*)
                FROM attendance_ggm a
                WHERE a.UserID = e.UserID
                  AND a.att_date BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            ELSE 0
        END AS biometric_present
    FROM emp e
),
working_days AS (
    SELECT 
        e.empname,
        CASE 
            WHEN e.work_location = 'Visakhapatnam' THEN (
                SELECT COUNT(*)
                FROM valid_dates_vskp v
                WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            WHEN e.work_location = 'Gurugram' THEN (
                SELECT COUNT(*)
                FROM valid_dates_ggm v
                WHERE v.dt BETWEEN GREATEST(e.empdoj, '2024-01-01') AND '2025-03-31'
            )
            ELSE 0
        END AS total_working_days
    FROM emp e
)

SELECT 
    e.empname,
    e.empdoj,
    e.work_location,
    (COALESCE(p.biometric_present, 0) + COALESCE(l.leave_days, 0)) AS present_days,
    COALESCE(w.total_working_days, 0) AS working_days
FROM emp e
LEFT JOIN present_days p ON e.empname = p.empname
LEFT JOIN leave_days l ON e.empname = l.empname
LEFT JOIN working_days w ON e.empname = w.empname;

        """

        cursor.execute(query)
        result = cursor.fetchall()

        conn.close()

        # Convert the result into a DataFrame for easy handling
        df = pd.DataFrame(result)
        return df

    except mysql.connector.Error as err:
        print(f"Database error: {err}")
        return pd.DataFrame()

# Fetch and display the results
df = first_feature()
df.head(25)
